### Data Prep

In [1]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('student_data.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

### Define Activation Function

In [2]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

### One Layer Neural Network
#### Set parameters

In [20]:
# Use same seed to make debugging easier
np.random.seed(42)

# Initialize weights
n_records, n_features = features.shape
weights = np.random.normal(scale = 1/n_features**.5, size = n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

#### Training
Train Neural Network and print the result 10 times

In [21]:
last_loss = None
for e in range(epochs):
    # For each iteration:
    # Step 1 - Calculate total weight change
    del_w = np.zeros(weights.shape) # initialize total weight change
    for x, y in zip(features.values, targets): # Loop through all records, x is the input, y is the target
        h = np.dot(weights, x) # the weighted sum
        output = sigmoid(h) # the prediction y_hat
        error = y - output # the error
        error_term = error*output*(1-output) # the error term
        # Calculate the change in weights for this sample and add it to the total weight change
        del_w += learnrate * error_term * x
    # Step 2 - Update weights
    weights += del_w / n_records
    
    # Print out the mean square error on the traning set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets)**2)
        if last_loss and last_loss < loss:
            print("Train loss: ",loss, " WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.26276093849966364
Train loss:  0.20928619409324895
Train loss:  0.20084292908073417
Train loss:  0.1986215647552789
Train loss:  0.19779851396686018
Train loss:  0.19742577912189863
Train loss:  0.19723507746241065
Train loss:  0.19712945625092465
Train loss:  0.19706766341315077
Train loss:  0.19703005801777368


#### Calculate accuracy on test data

In [22]:
test_out = sigmoid(np.dot(features_test, weights))
predictions = test_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.725


### Two Layer Neural Network
#### Set parameters

In [3]:
# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape

#### Training
Train Neural Network and print the result 10 times

In [4]:
last_loss = None

# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))

        ## Backward pass ##
        # Calculate the network's prediction error
        error = y - output

        # Calculate error term for the output unit
        output_error_term = error * output * (1 - output)

        ## propagate errors to hidden layer

        # Calculate the hidden layer's contribution to the error
        hidden_error = weights_hidden_output * output_error_term 
        
        # Calculate the error term for the hidden layer
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output)
        
        # Update the change in weights
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:,None]

    # Update weights  (division by n_records or number of samples)
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss



Train loss:  0.23143745682765984
Train loss:  0.23114026554406467
Train loss:  0.2308513127837036
Train loss:  0.23057034878009597
Train loss:  0.23029713165262303
Train loss:  0.23003142717151226
Train loss:  0.2297730085275678
Train loss:  0.22952165610674327
Train loss:  0.22927715726974604
Train loss:  0.22903930613669263


#### Calculate accuracy on test data

In [5]:
# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))


Prediction accuracy: 0.750
